In [119]:
from pysolar.solar import get_altitude, get_azimuth
import matplotlib.image as mpimg
from datetime import datetime, timedelta
from pytz import timezone
import numpy as np

def calculate_solar_positions(
    start_date: datetime,
    start_hour: int = 10,
    end_hour: int = 17,
    latitude: float = -0.2105367,
    longitude: float = -78.491614
):
    """Calcula posiciones solares y ángulos para una fecha específica y rango de horas.

    ## Parameters
    - start_date: Fecha y hora de inicio para la simulación (se usa solo la fecha, hora se define con start_hour).
    - start_hour: Hora de inicio del cálculo.
    - end_hour: Hora de fin del cálculo.
    - latitude: Latitud para la posición geográfica.
    - longitude: Longitud para la posición geográfica.

    ## Return
    - times: Lista de tiempos de simulación.
    - azimuths: Lista de ángulos azimutales.
    - elevations: Lista de ángulos de elevación.
    - beta: Lista de ángulos de rollo calculados.
    - phi: Lista de ángulos de inclinación calculados.
    """
    times = []
    azimuths = []
    elevations = []
    beta = []
    phi = []

    time_interval = timedelta(hours=0.33)

    # Crear el rango de tiempos basado en la hora de inicio y fin
    start_time = start_date.replace(hour=start_hour, minute=0, second=0, microsecond=0)
    end_time = start_date.replace(hour=end_hour, minute=0, second=0, microsecond=0)

    current_time = start_time
    while current_time <= end_time:
        az = get_azimuth(latitude, longitude, current_time)
        el = get_altitude(latitude, longitude, current_time)

        # Convertir a radianes para los cálculos
        az_rad = (az * np.pi) / 180
        el_rad = (el * np.pi) / 180

        val = np.cos(el_rad) * np.sin(az_rad)
        beta_rad = np.arcsin(val)
        beta_deg = (beta_rad * 180) / np.pi

        val_fi1 = -(np.cos(el_rad) * np.cos(az_rad)) / np.cos(beta_rad)
        x_rad = np.arcsin(val_fi1)
        x_deg = (x_rad * 180) / np.pi

        val_fi2 = np.sin(el_rad) / np.cos(beta_rad)
        y_rad = np.arccos(val_fi2)
        y_deg = (y_rad * 180) / np.pi

        times.append(current_time)
        azimuths.append(az)
        elevations.append(el)
        beta.append(beta_deg)
        phi.append(x_deg)  # Asumiendo que phi corresponde a x_deg; ajusta si es necesario

        current_time += time_interval

    return times, azimuths, elevations, beta, phi

In [123]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
import numpy as np

# Extraer los datos de tiempo, azimuth y elevación
#times = [pos[0] for pos in solar_positions]
#azimuths = [pos[1] for pos in solar_positions]
#elevations = [pos[2] for pos in solar_positions]
#beta = [sol[0] for sol in solucion]
#phi = [sol[1] for sol in solucion]

fecha = input("Ingrese una fecha en formato año-mes-día (ejemplo: 2024-08-11): ")
anio, mes, dia = fecha.split('-')
rango = input("Ingrese el rango de la simulación (ejemplo: 10-17): ")
inicio, fin = rango.split('-')

start_date = datetime(int(anio), int(mes), int(dia), tzinfo=timezone("America/Guayaquil"))
times, azimuths, elevations, beta, phi = calculate_solar_positions(start_date, start_hour=int(inicio), end_hour=int(fin))

In [124]:
# Supongamos que ya has calculado los valores de times, azimuths, elevations, beta, y phi
# Aquí está el código para imprimirlos en un formato legible.

print("Solar Position Data:\n")

print("Times: ", end="")
print(", ".join([f"{time.strftime('%H:%M')}" for time in times]))

print("\nAzimuths: ", end="")
print(", ".join([f"{azimuth:.2f}°" for azimuth in azimuths]))

print("\nElevations: ", end="")
print(", ".join([f"{elevation:.2f}°" for elevation in elevations]))

print("\nBeta (Ángulo de Inclinación): ", end="")
print(", ".join([f"{b:.2f}°" for b in beta]))

print("\nPhi (Ángulo de Orientación): ", end="")
print(", ".join([f"{p:.2f}°" for p in phi]))

Solar Position Data:

Times: 08:00, 08:19, 08:39, 08:59, 09:19, 09:39, 09:58, 10:18, 10:38, 10:58, 11:18, 11:37, 11:57, 12:17, 12:37, 12:57, 13:16, 13:36, 13:56, 14:16, 14:36, 14:55, 15:15, 15:35, 15:55, 16:15, 16:34, 16:54

Azimuths: 63.22°, 61.85°, 60.16°, 58.07°, 55.49°, 52.29°, 48.31°, 43.36°, 37.19°, 29.58°, 20.43°, 9.91°, 358.60°, 347.41°, 337.19°, 328.42°, 321.19°, 315.34°, 310.64°, 306.88°, 303.84°, 301.39°, 299.41°, 297.80°, 296.51°, 295.48°, 294.67°, 294.06°

Elevations: 28.37°, 32.76°, 37.08°, 41.33°, 45.47°, 49.47°, 53.27°, 56.83°, 60.03°, 62.76°, 64.86°, 66.16°, 66.53°, 65.92°, 64.41°, 62.14°, 59.28°, 55.98°, 52.36°, 48.50°, 44.46°, 40.29°, 36.02°, 31.68°, 27.28°, 22.84°, 18.37°, 13.87°

Beta (Ángulo de Inclinación): 51.77°, 47.86°, 43.79°, 39.59°, 35.30°, 30.94°, 26.52°, 22.06°, 17.57°, 13.06°, 8.53°, 3.99°, -0.56°, -5.10°, -9.64°, -14.17°, -18.67°, -23.16°, -27.61°, -32.01°, -36.36°, -40.63°, -44.79°, -48.83°, -52.69°, -56.30°, -59.59°, -62.43°

Phi (Ángulo de Orientació

In [129]:
x_labels = []
for time in times:
        x_labels.append(f"{time.hour}:{time.minute}")
x_labels

['8:0',
 '8:19',
 '8:39',
 '8:59',
 '9:19',
 '9:39',
 '9:58',
 '10:18',
 '10:38',
 '10:58',
 '11:18',
 '11:37',
 '11:57',
 '12:17',
 '12:37',
 '12:57',
 '13:16',
 '13:36',
 '13:56',
 '14:16',
 '14:36',
 '14:55',
 '15:15',
 '15:35',
 '15:55',
 '16:15',
 '16:34',
 '16:54']

In [135]:
# Cargar la imagen del sol
sun_img = mpimg.imread('./Interfaz-SeguidorSolar/images/sol4.jpg')

# Convertir los tiempos a números (por simplicidad en la gráfica 3D)
time_numbers = np.arange(len(times))

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica 3D
frame_3d = tk.Frame(root)
frame_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje 3D
fig3 = plt.Figure(figsize=(13, 7), dpi=100)
ax3 = fig3.add_subplot(111, projection='3d')

# Configurar los ejes
ax3.set_xlabel('Time')
ax3.set_ylabel('Azimuth')
ax3.set_zlabel('Elevation')
ax3.set_title('Trayectoria del sol')
ax3.legend()

# Crear una función para generar la esfera que representa el sol
def create_sun(ax, center, radius=0.5, color='yellow'):
    # Crear una esfera 3D
    u = np.linspace(0, 2 * np.pi, 30)
    v = np.linspace(0, np.pi, 15)
    x = radius * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = radius * np.outer(np.sin(u), np.sin(v)) + center[1]
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]

    # Añadir la esfera al eje 3D
    ax.plot_surface(x, y, z, color=color, shade=True)

# Función de inicialización para la animación 3D
def init_3d():
    ax3.set_xlim(min(time_numbers), max(time_numbers))
    ax3.set_ylim(min(azimuths), max(azimuths))
    ax3.set_zlim(min(elevations), max(elevations))
    return []

# Función de actualización para la animación 3D
def update_3d(frame):
    ax3.cla()  # Limpiar el eje antes de actualizar
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Azimuth')
    ax3.set_zlabel('Elevation')
    ax3.set_title('Trayectoria del sol')

    # Re-dibujar la trayectoria hasta el punto actual
    ax3.plot(time_numbers[:frame+1], azimuths[:frame+1], elevations[:frame+1], 'yo-', markersize=5)

    # Crear la esfera 3D para representar el sol en la posición actual
    sun_position = [time_numbers[frame], azimuths[frame], elevations[frame]]
    create_sun(ax3, sun_position, radius=0.5, color='yellow')

    return []

# Crear el lienzo para la figura 3D
canvas3 = FigureCanvasTkAgg(fig3, master=frame_3d)
canvas3.draw()
canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani3 = animation.FuncAnimation(fig3, update_3d, frames=len(times), init_func=init_3d, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()

C:\Users\Enrique_P\AppData\Local\Temp\ipykernel_8560\1715544158.py:24: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax3.legend()


In [111]:
# Convertir los azimuths de grados a radianes para el gráfico polar
azimuths_rad = np.radians(azimuths)

# Cargar la imagen del sol
sun_img = mpimg.imread('./Interfaz-SeguidorSolar/images/sol4.jpg')

# Configurar la ventana de Tkinter
root = tk.Tk()
root.title("Simulación de Ángulos Solares")

# Crear un marco para la gráfica polar
frame_polar = tk.Frame(root)
frame_polar.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Crear la figura y el eje polar
fig_polar = plt.Figure(figsize=(13, 7), dpi=100)
ax_polar = fig_polar.add_subplot(111, projection='polar')

# Configurar la dirección del ángulo (en sentido horario y con 0° en la parte superior)
ax_polar.set_theta_direction(-1)
ax_polar.set_theta_offset(np.pi / 2.0)

# Configurar los ejes
ax_polar.set_title('Trayectoria del sol en Coordenadas Polares')

# Función de inicialización para la animación polar
def init_polar():
    ax_polar.set_ylim(min(elevations), max(elevations))
    return []

# Función de actualización para la animación polar
def update_polar(frame):
    ax_polar.clear()  # Limpiar el gráfico en cada frame
    ax_polar.set_theta_direction(-1)
    ax_polar.set_theta_offset(np.pi / 2.0)
    ax_polar.set_ylim(min(elevations), max(elevations))
    ax_polar.set_title('Trayectoria del sol en Coordenadas Polares')
    
    theta = azimuths_rad[frame]  # Ángulo actual en radianes
    r = elevations[frame]  # Elevación actual
    
    # Superponer la imagen del sol en la ubicación actual
    ax_polar.imshow(sun_img, extent=(theta - 0.1, theta + 0.1, r - 0.1, r + 0.1), zorder=5)
    
    # Para mantener la trayectoria dibujada, cambiar color a rojo ('r') y el marcador a un punto verde ('go-')
    ax_polar.plot(azimuths_rad[:frame+1], elevations[:frame+1], color='red', marker='o', markerfacecolor='yellow', linestyle='-', markersize=5, label='Trayectoria del Sol')

    return []

# Crear el lienzo para la figura polar
canvas_polar = FigureCanvasTkAgg(fig_polar, master=frame_polar)
canvas_polar.draw()
canvas_polar.get_tk_widget().pack(fill=tk.BOTH, expand=True)

interval_time_ms = 500
ani_polar = animation.FuncAnimation(fig_polar, update_polar, frames=len(times), init_func=init_polar, interval=interval_time_ms, blit=True)

# Iniciar la interfaz gráfica de Tkinter
root.mainloop()